In [1]:
import os
import pandas as pd

!pwd
!ls

/mnt/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw
 ana-flavia-HSD-NCxHSD-NR.tsv
 ana-flavia-NCxNR.tsv
 ana-flavia-NCxSTD-NC.tsv
'ana-flavia-samples-groups - Copia.tsv'
 ana-flavia-samples-groups.tsv
 ana-flavia-STD-NCxHSD-NC.tsv
 ana-flavia-STD-NCxSTD-NR.tsv
 ana-flavia-STD-NRxHSD-NR.tsv
 filtered-metadata-all-ana.tsv
 filter_samples.ipynb
 karina-samples-groups.csv
 manifest
 manifest-all-ana.csv
 manifest-all.csv
 manifest-all.tsv
 metadata
 metadata-all-ana.tsv
 metadata-all-ana-types.tsv
 preprocess-ana-flavia-sample-data.ipynb
 process-manifest-prefix-path.ipynb
 sample_id-sample_name.tsv
 samples-paths.tsv


In [2]:
samples_info = "ana-flavia-samples-groups.tsv"
metadata_all = "metadata-all-ana.tsv"
filtered_metadata_all = "filtered-metadata-all-ana.tsv"

In [3]:
# Extract mmanifest from CSV
samples_df = pd.read_csv(samples_info, sep='\t')
print(samples_df.head())

metadata_df = pd.read_csv(metadata_all, sep='\t')
print(metadata_df.head())

  sample-id group-id        group-desc
0     N9.01       NC  Ninhada controle
1    N11.01       NC  Ninhada controle
2     N9.05       NC  Ninhada controle
3    N11.02       NC  Ninhada controle
4     N9.03       NR  Ninhada reduzida
       sample-id sample-name group-id        group-desc
0  S210421121673       N9.01       NC  Ninhada controle
1  S210421121674      N11.01       NC  Ninhada controle
2  S210421121675       N9.05       NC  Ninhada controle
3  S210421121676      N11.02       NC  Ninhada controle
4  S210421121677       N9.03       NR  Ninhada reduzida


In [4]:
s1 = metadata_df.loc[:,'sample-name'].values
s2 = samples_df.loc[:,'sample-id'].values

len(set(s1) & set(s2))

38

In [5]:
tmp_df = metadata_df.groupby(by='sample-name').count() > 1
repeated_idx = tmp_df[tmp_df['sample-id']].index
repeated_idx

Index(['N1.03', 'N1.04', 'N2.02'], dtype='object', name='sample-name')

In [6]:
metadata_df[metadata_df.loc[:,'sample-name'].isin(repeated_idx)]

,sample-id,sample-name,group-id,group-desc
25,S210421121698,N1.04,HSD-NC,Grupo dieta high-sugar
26,S210421121711,N1.04,HSD-NC,Grupo dieta high-sugar
32,S210421121704,N1.03,STD-NR,Grupo dieta padrão
33,S210421121713,N1.03,STD-NR,Grupo dieta padrão
34,S210421121705,N2.02,STD-NR,Grupo dieta padrão
35,S210421121714,N2.02,STD-NR,Grupo dieta padrão


In [7]:
# Select fisrt occurency of each 
filtered_ids = metadata_df.groupby('sample-name').first()['sample-id']
filtered_ids.values

array(['S210421121694', 'S210421121703', 'S210421121704', 'S210421121698',
       'S210421121707', 'S210421121678', 'S210421121679', 'S210421121680',
       'S210421121681', 'S210421121674', 'S210421121676', 'S210421121705',
       'S210421121706', 'S210421121699', 'S210421121708', 'S210421121695',
       'S210421121700', 'S210421121709', 'S210421121710', 'S210421121696',
       'S210421121697', 'S210421121701', 'S210421121702', 'S210421121682',
       'S210421121683', 'S210421121685', 'S210421121686', 'S210421121688',
       'S210421121691', 'S210421121692', 'S210421121684', 'S210421121687',
       'S210421121689', 'S210421121690', 'S210421121693', 'S210421121673',
       'S210421121677', 'S210421121675'], dtype=object)

In [8]:
filter_mask = metadata_df.loc[:, 'sample-id'].isin(filtered_ids.values).values
filter_mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True])

In [9]:
filtered_metadata_df = metadata_df[filter_mask].reset_index(drop=True)
filtered_metadata_df.describe()

,sample-id,sample-name,group-id,group-desc
count,38,38,38,38
unique,38,38,6,4
top,S210421121673,N9.01,HSD-NC,Grupo dieta high-sugar
freq,1,1,8,15


In [10]:
filtered_metadata_df.to_csv(filtered_metadata_all, index=False, sep='\t')